Question 1)

In [ ]:

def s(n):
    'function with input n, output is the sum of all divisors of n (except n)'
    
    #list of divisors
    divisors = []

    #checks every number up to n/2 to see if they are divisors, and if they are, add to the divisors list - we only go up to n/2 because anything larger can't divide n, so this is the upper bound
    for i in range(1, (n//2) + 1):
        if (n % i) == 0:
            divisors.append(i)
            
    #returns the sum of all divisors of n, except for n itself.
    return sum(divisors)


In [ ]:
#Example to test code
s(102)

Question 2)

In [ ]:
def a(n, iterations):
    'function that gives the aliquot sequence of s(n). Inputs are n (the starting number), and iterations (number of elements in the sequence)'
    
    #the aliquot sequence of s(n) as a list, which is also the output.
    aliquot_seq = [n]

    #already placed the first term (n) in the list, so you only need to generate up to iterations - 1 more terms.
    for i in range(iterations - 1):
        
        #if s(n) = 0 then the sequence terminates.
        if s(n) == 0: 
            aliquot_seq.append(s(n))
            return aliquot_seq

        #if s(n) > 10^9 then the sequence terminates    
        elif s(n) > 10**9:
            return aliquot_seq

        #adds the current s(n) to the aliquot sequence and changes the input value to s(n)    
        else:
            aliquot_seq.append(s(n))
            n =s(n)    
            
    return aliquot_seq

In [ ]:
a(1000, 30)

Question 3)

In [ ]:
def detect_loop(n):

    #We want to create a list of the values in the aliquot sequence for n so we know if a value repeats
    repeats = [n]
    #Print the first value n
    print(n)


    #Compute the next term by using the previously defined function s(n)
    next_term = s(n)

    #We then create a loop for when the next term in the sequence is NOT repeated
    while next_term not in repeats:
        #If the next term is not in our repeats list, we print this value
        print(next_term)    
        #We then add this term to our repeats list 
        repeats.append(next_term)
        #We then make our current term the term we just used
        n = next_term
        #We then compute the next term using function s(n) and then repeat this loop until a term repeats
        next_term = s(n)

    #A term has repeated so it exits the loop and prints the repeating term
    print("The sequence now loops")

In [ ]:
detect_loop(562)

Question 4)

In [ ]:
#Either 1)Terminates at 0 2)Enters a loop 3)Sequence cut short because there's more than k terms in sequence or 4)Sequence cut short because a value of a term has exceeded i

#We first want to determine the ending of the different aliquot sequences
def sequence_end(n, k, cutoff):
    
    #This is again for the loops
    repeats = {n}

    #Generate k terms 
    for i in range(k - 1):

        #Want to find the next term in the sequence
        next_term = s(n)

        #Ending 1 - terminates at 0
        if next_term == 0:
            return "zero"

        #Ending 2 - term exceeds i
        if next_term > cutoff:
            return "cutoff"

        #Ending 3 - term repeats
        if next_term in repeats:
            return "loop"

        #If term NOT in 'repeats' - add this term to set of repeats and continue sequence
        repeats.add(next_term)
        n = next_term

    #Ending 4 - greater than k terms - has to be this if it's not the others?     
    return "k_limit"

#NOw we want to count how many sequences are in each category of ending
def count_reason(m, k, cutoff):

    #Starting at 0 for each
    zero = loop = cutoff_count = k_limit = 0

    #We want to test for each n up to m 
    for n in range(0, m):

        #How does the sequence end?
        reason = sequence_end(n, k, cutoff)

        #We want to increase the relevant category
        if reason == "zero":
            zero = zero + 1
        elif reason == "loop":
            loop = loop + 1
        elif reason == "cutoff":
            cutoff_count = cutoff_count + 1
        elif reason == "k_limit":
            k_limit = k_limit + 1

    print('Number of terminations of each category for n < ' + str(m))
    print('Terminates at 0: ' + str(zero))
    print('Enters a loop: ' + str(loop))
    print('Exceeds 10^9: ' + str(cutoff_count))
    print('Hits more than 30 terms: ' + str(k_limit))

count_reason(1000, 30, 10**9)